In [79]:
import pandas as pd
import numpy as np
from tqdm import tqdm

import gurobipy as gp
from gurobipy import GRB

In [80]:
macrotick = 100
sync_error = 0
time_out = 4 * 60 * 60

NUM_FLOW = 18
DATA_NAME = "harmonic12"
TOPO_NAME = "0"

task = pd.read_csv("../../dac_data/%s.csv"%DATA_NAME)[:NUM_FLOW]
network = pd.read_csv("../../dac_data/%s_topology.csv"%TOPO_NAME)
for col in ['size','period','deadline','jitter']:
    task[col] = np.ceil(task[col] / macrotick).astype(int)
for col in ['t_proc','t_prop']:
    network[col] = np.ceil(network[col] / macrotick).astype(int)
    
nodes = list(network['link'].apply(lambda x:eval(x)[0])) + \
    list(network['link'].apply(lambda x:eval(x)[1]))
NODE_SET = list(set(nodes))
ES_set = [x for x in NODE_SET if nodes.count(x) == 2]
SW_set = list(set(NODE_SET) - set(ES_set))
LCM = np.lcm.reduce(task['period'])
net = np.zeros(shape = (max(NODE_SET) + 1, max(NODE_SET) + 1))

In [81]:
'''
There is a bug in the overlap constraints that causes overlap when window-start of f_j is set to x + 1 and window-close of i is set to x. 
The bug is fixed when we change x+1 to x+5.
'''

'\nThere is a bug in the overlap constraints that causes overlap when window-start of f_j is set to x + 1 and window-close of i is set to x. \nThe bug is fixed when we change x+1 to x+5.\n'

## 1. Model

In [82]:
M = int(1e16)

In [83]:
m = gp.Model("RTCSA2020")

Network model

$I=(V, E, D)$

In [84]:
net_var = {}

for _, row in network.iterrows():
    net_var.setdefault(eval(row['link'])[0], {})
    net_var[eval(row['link'])[0]]['msd'] = row['t_proc']
    net[eval(row['link'])[0], eval(row['link'])[1]] = 1

## Create mapping from Link to index
link_to_index = {}
index_to_link = {}

counter = 0
for _, row in network.iterrows():
    link = row['link']
    link_to_index[link] = counter
    index_to_link[counter] = link
    counter += 1


Task model



\begin{equation}
\begin{array}{ll}
p_{k m} & \text { binary: } 1 \text { if flow } f_{k} \text { is routed via link } m \in E \\
a_{k l m u v} & \text { binary: } 1 \text { if } f_{k} \text { is scheduled before } f_{l} \text { on } m(\mathrm{cf} .(7) \text { and (8)) } \\
o_{k m} & \text { integer: number of offset cycles for transmission start of } f_{k} \text { on } m \\
t_{k m} & \text { integer: transmission start of } f_{k} \text { on } m \text { within interval }\left[0, f_{k} \cdot c t-1\right] \\
\hline
\end{array}
\end{equation}

In [85]:
p = m.addMVar(shape=(len(task), len(link_to_index)), vtype=GRB.BINARY , name="routing")
t = m.addMVar(shape=(len(task), len(link_to_index)), vtype=GRB.INTEGER , name="time_start")

$$f_{k}=\left(src_{k}, dst_{k}, ct_{k}, rsl_{k}, ml_{k}\right)$$

In [86]:
task_attr = {}
for k, row in task.iterrows():
#     task_var.setdefault(k, {})
    task_attr.setdefault(k, {})
    
    ## f_k
    task_attr[k]['src'] = int(row['src'])
    task_attr[k]['dst'] = eval(row['dst']) if isinstance(row['dst'], str) else [row['dst']]
    task_attr[k]['ct'] = int(row['period'])
    task_attr[k]['rsl'] = int(row['size'] * 8)
    task_attr[k]['ml'] = int(row['deadline'])
    
    ## Bound the t matrix
    for j in index_to_link:
        m.addConstr(t[k][j] <= task_attr[k]['ct'] - task_attr[k]['rsl'])
        
#     for _, row in network.iterrows():
#         link = row['Link']
#         task_var[k].setdefault(link, {})
#         ## Routing
#         task_var[k][link]['route'] = m.addVar(vtype=GRB.BINARY , name="x_%d_%s"%(k, link))
#         ## Scheduling
#         task_var[k][link]['flow_start'] = m.addVar(vtype=GRB.INTEGER , name="t_%d_%s"%(k, link))
#         task_var[k][link]['flow_offset'] = m.addVar(vtype=GRB.INTEGER , name="o_%d_%s"%(k, link))

## 2. Route preprocessing

In [87]:
link_in = {}
link_out = {}
for link in link_to_index.keys():
    link = eval(link)
    link_in.setdefault(link[1], [])
    link_in[link[1]].append(str(link))
    link_out.setdefault(link[0], [])
    link_out[link[0]].append(str(link))

In [88]:
vk = {}
for k in task_attr:
    vk[k] = []
    vk[k] += list(SW_set)
    vk[k] += list([task_attr[k]['src']])
    vk[k] += task_attr[k]['dst']

## 3. Constraints

### 3.1 Multicast routing

$\forall k \in F, \forall i \in f_{k} . d s t s:$
$$
\sum_{\bar{m} \in \breve{E}_{k i}} p_{k \bar{m}}=1
$$

In [89]:
for k in task_attr:
    for i in task_attr[k]['dst']:
        m.addConstr(
            gp.quicksum(
               p[k][link_to_index[link]] for link in link_in[i]
            ) == 1
        )
    for v in ES_set:
        m.addConstr(
            gp.quicksum(p[k][link_to_index[link]] for link in link_in[v]
                        if v not in task_attr[k]['dst']) == 0)

## Have to specific the source
for k in task_attr:
    m.addConstr(
        gp.quicksum(
               p[k][link_to_index[link]] for link in link_out[task_attr[k]['src']]
        ) == 1
    )
    m.addConstr(
        gp.quicksum(
               p[k][link_to_index[link]] for link in link_in[task_attr[k]['src']]
        ) == 0
    )
    for v in ES_set:
        m.addConstr(
            gp.quicksum(
                   p[k][link_to_index[link]] for link in link_out[v] if v != task_attr[k]['src']
            ) == 0
        )


\begin{equation}
\begin{array}{l}
\forall k \in F, \forall i \in V_{k} \backslash\left\{f_{k} . s r c\right\}: \\
\sum_{\widehat{m} \in \widehat{E}_{k i}} p_{k \widehat{m}} \leqslant \sum_{\widetilde{m} \in \breve{E}_{k i}} p_{k \widetilde{m}} \cdot M_{2}
\end{array}
\end{equation}

In [90]:
for k in task_attr:
    for i in set(vk[k]) - set([task_attr[k]['src']]):
        m.addConstr(
            gp.quicksum(
                p[k][link_to_index[link]] for link in link_out[i]
            )
            <=
            gp.quicksum(
                p[k][link_to_index[link]] for link in link_in[i]
            ) * M
        )
#     ## This constraint is required to prune the paths ending in switches
#     for i in set(vk[k]) - set(task_attr[k]['dst']):
#         m.addConstr(
#             gp.quicksum(
#                 p[k][link_to_index[link]] for link in link_in[i]
#             )
#             <=
#             gp.quicksum(
#                 p[k][link_to_index[link]] for link in link_out[i]
#             )
#         )

\begin{aligned}
&\forall k \in F, \forall i \in V_{k} \backslash\left\{f_{k} \cdot s r c\right\}, \forall \widehat{m} \in \widehat{E}_{k i}: \\
&p_{k \widehat{m}} \leqslant \sum_{\widetilde{m} \in \breve{E}_{k i} \mid \widetilde{m}_{0} \neq \widehat{m}_{1}} p_{k \widetilde{m}}
\end{aligned}

In [91]:
# for k in task_attr:
#     for i in set(vk[k]) - set([task_attr[k]['src']]):
#         for m_out in link_out[i]:
#             p[k][link_to_index[m_out]] <= gp.quicksum(
#                 p[k][link_to_index[link]] for link in link_in[i] if eval(link)[0] != eval(m_out)[1]
#             )

### 3.2 Multicast Path Scheduling

$$t_{-} d_{k m n}=\left(o_{k n} \cdot f_{k} . c t+t_{k n}\right)-\left(o_{k m} \cdot f_{k} . c t+t_{k m}\right)$$

Intuitive Path Scheduling

\begin{array}{r}
\forall k \in F, \forall i \in V_{k} \backslash\left\{f_{k} . s r c\right\}, \forall \widehat{m} \in \widehat{E}_{k i}, \forall \check{m} \in \check{E}_{k i}: \\
t_{-} d_{k \bar{m} \widehat{m}} \geqslant v_{i} . m s d-M_{4} \cdot\left(1-p_{k \widetilde{m}}\right)
\end{array}

In [92]:
for k in task_attr:
    for i in set(vk[k]) - set([task_attr[k]['src']]):
        for m_out in link_out[i]:
            m_out = link_to_index[m_out]
            for m_in in link_in[i]:
                m_in = link_to_index[m_in]
                m.addConstr(
                    (t[k][m_out]) - 
                    (t[k][m_in]) >=
                    net_var[i]['msd'] + task_attr[k]['rsl'] - M * (1 - p[k][m_out])
                )
                

Shortest Path Consideration

\begin{gathered}
\forall k \in F, \forall \hat{m} \in \widehat{E}_{k f_{k} . s r c}, \forall m \in E_{k} \mid m_{0} \neq f_{k} . s r c: \\
t_{-} d_{k \widehat{m} m} \geqslant \sum_{i \in S P\left(f_{k} \cdot s r c, m_{0}\right)} v_{i} \cdot m s d
\end{gathered}

In [93]:
## Shortest path
def bfs_paths(graph, start, goal):
    queue = [(start, [start])]
    while queue:
        (vertex, path) = queue.pop(0)
        for _next in set(np.reshape(np.argwhere(graph[vertex] > 0),  -1)) - set(path):
            if _next == goal:
                yield path + [_next]
            else:
                queue.append((_next, path + [_next]))

In [94]:
# for k in task_attr:
#     for m_out in link_out[task_attr[k]['src']]:
#         for link in [link for link in link_to_index if eval(link)[0] != task_attr[k]['src']]:
#             m.addConstr(
#                 (t[k][link_to_index[link]]) - 
#                 (t[k][link_to_index[m_out]]) >=
#                 gp.quicksum([net_var[v]['msd'] + task_attr[k]['rsl'] for v in next(bfs_paths(net, task_attr[k]['src'], eval(link)[0]))[:-1]])
#             )

Path Dependency Reduction

$$t_{-} d_{k \widetilde{m} \widehat{m}} \geqslant v_{i} . m s d$$

In [95]:
# for k in tqdm(task_attr):
#     for i in range(NUM_SW):
#         for m_in in link_in[i]:
#             for m_out in link_out[i]:
#                 m.addConstr(
#                     t[k][link_to_index[m_out]] - 
#                     t[k][link_to_index[m_in]] >=
#                     net_var[v]['msd'] + task_attr[k]['rsl']
#                 )

### 3.3 Resource Constraints

\begin{aligned}
&\forall(k, l) \in F^{2} \mid l>k, \forall m \in E_{k} \cap E_{l} \text {, }\\
&\forall(u, v) \in \mathbb{N}^{2} \mid u \leqslant \frac{l c m\left(f_{k} \cdot c t, f_{l} \cdot c t\right)}{f_{k} \cdot c t} \wedge v \leqslant \frac{l c m\left(f_{k} \cdot c t, f_{l} . c t\right)}{f_{l} . c t} \text { : }\\
&\left(t_{l m}+v \cdot f_{l} . c t\right)-\left(t_{k m}+u \cdot f_{k} . c t\right)\\
&\geqslant f_{k} \cdot r s l-M_{7} \cdot\left(3-a_{k l m u v}-p_{k m}-p_{l m}\right)\\
&\left(t_{k m}+u \cdot f_{k} . c t\right)-\left(t_{l m}+v \cdot f_{l} . c t\right)\\
&\geqslant f_{l} \cdot r s l-M_{8} \cdot\left(2+a_{k l m u v}-p_{k m}-p_{l m}\right)
\end{aligned}

In [96]:
for k, l in [(k, l) for k in task_attr for l in task_attr if k < l]:
    for link in index_to_link:
        ctl, ctk = int(task_attr[l]['ct']), int(task_attr[k]['ct'])
        t_ijl, t_ijk = t[l][link], t[k][link]
        rsl_k, rsl_l = task_attr[k]['rsl'], task_attr[l]['rsl']
        for u, v in [(u, v) 
                         for u in range(0, int(np.lcm(ctk, ctl) / ctk)) 
                         for v in range(0, int(np.lcm(ctk, ctl) / ctl))]:
            _inte = m.addVar(vtype= GRB.BINARY, name = "%s%d%d%d%d"%(link, k, l, u, v))
            m.addConstr((t_ijl + v * ctl) - (t_ijk + u * ctk) 
                        >= rsl_k - M * (3 - _inte - p[k][link] - p[l][link])) 
            m.addConstr((t_ijk + u * ctk) - (t_ijl + v * ctl)
                       >= rsl_l - M * (2 + _inte - p[k][link] - p[l][link])) 

### 3.4 Application Constraints

$$l t_{k i}=\sum_{\bar{m} \in \breve{E}_{k i}}\left(o_{k \bar{m}} \cdot f_{k} \cdot c t+t_{k \bar{m}}\right)-\sum_{\widehat{m} \in \widehat{E}_{k f_{k} \cdot s r c}}\left(o_{k \widehat{m}} \cdot f_{k} \cdot c t+t_{k \widehat{m}}\right)$$

$\forall k \in F, \forall i \in f_{k} . d s t s:$
$$
l t_{k i}+f_{k} \cdot r s l \leqslant f_{k} \cdot m l
$$

In [97]:
for k in task_attr:
    for i in task_attr[k]['dst']:
        m.addConstr(
            gp.quicksum(t[k][link_to_index[link]] for link in link_in[i])
            - 
            gp.quicksum(t[k][link_to_index[link]] for link in link_out[task_attr[k]['src']])
            + task_attr[k]['rsl'] + net_var[eval(link_in[i][0])[0]]['msd'] <= task_attr[k]['ml']
        )

## 4.Optimization

Flow Latency Minimization

$$\operatorname{Minimize} \sum_{k \in F} \sum_{i \in f_{k} . d s t s} l t_{k i}$$

In [98]:
# m.setObjective(
#     gp.quicksum(
#         gp.quicksum(
#             gp.quicksum(o[k][link_to_index[link]] * task_attr[k]['ct']
#                         + t[k][link_to_index[link]] for link in link_in[i])
#             - 
#             gp.quicksum(o[k][link_to_index[link]] * task_attr[k]['ct']
#                         + t[k][link_to_index[link]] for link in link_out[task_attr[k]['src']])
#             for i in task_attr[k]['dst']
#         )
        
#     for k in task_attr
#     )
#     ,GRB.MINIMIZE)

Shortest Path with Secondary Latency Minimization

$$\text { Minimize } \sum_{k \in F} \sum_{m \in E_{k}} p_{k m}$$

In [99]:
# m.setObjective(
#     gp.quicksum(
#         gp.quicksum(
#             p[k][link_to_index[m]]
#             for m in link_to_index
#         )
        
#     for k in task_attr
#     )
#     ,GRB.MINIMIZE)

In [100]:
try: 
    m.optimize()
except gp.GurobiError as E:
    print("Optimize failed", E)

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[arm])
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads
Optimize a model with 24696 rows, 12330 columns and 117612 nonzeros
Model fingerprint: 0x08c17531
Variable types: 0 continuous, 12330 integer (11790 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+16]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+16]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 23565 rows and 11726 columns
Presolve time: 0.13s
Presolved: 1131 rows, 604 columns, 3921 nonzeros
Variable types: 0 continuous, 604 integer (489 binary)
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.15 seconds (0.29 work units)
Thread count was 10 (of 10 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.00000000000

In [101]:
m.status

2

## Output schedule

In [102]:
queue_count = {}
queue_log = {}
GCL = []
for i in task_attr:
    for e in [index_to_link[ei] for ei, x in enumerate(p[i]) if x.x > 0]:
        queue_count.setdefault(e, 0)
        start = t[i][link_to_index[e]].x
        end = start + task_attr[i]['rsl']
        queue = queue_count[e]
        p_task = task.loc[i, 'period']
        for k in range(int(LCM / p_task)):
            GCL.append(
                [eval(e), queue, int(start + k * p_task) * macrotick, int(end + k * p_task) * macrotick, LCM * macrotick]
            )
        queue_log[(i, e)] = queue
        queue_count[e] += 1

In [103]:
OFFSET = []
for i in task_attr:
    for e in [index_to_link[ei] for ei, x in enumerate(p[i]) if x.x > 0]:
        if eval(e)[0] == task_attr[i]['src']:
            OFFSET.append(
                [i, 0, (task.loc[i, 'period'] - t[i][link_to_index[e]].x) * macrotick]
            )

In [104]:
ROUTE = []
for i, row in task.iterrows():
    for link in [index_to_link[ei] for ei, x in enumerate(p[i]) if x.x > 0]:
        ROUTE.append([i, link])

In [105]:
QUEUE = []
for i in task_attr:
    for e in [index_to_link[ei] for ei, x in enumerate(p[i]) if x.x > 0]:
        QUEUE.append([i, 0, e, queue_log[(i, e)]])

In [106]:
GCL = pd.DataFrame(GCL)
GCL.columns = ["link", "queue", "start", "end", "cycle"]
GCL.to_csv("RTCSA2020-%s-%d-%s-GCL.csv"%(DATA_NAME,NUM_FLOW,TOPO_NAME), index=False)

OFFSET = pd.DataFrame(OFFSET)
OFFSET.columns = ['id', 'ins_id', 'offset']
OFFSET.to_csv("RTCSA2020-%s-%d-%s-OFFSET.csv"%(DATA_NAME,NUM_FLOW,TOPO_NAME), index=False)

ROUTE = pd.DataFrame(ROUTE)
ROUTE.columns = ['id', 'link']
ROUTE.to_csv("RTCSA2020-%s-%d-%s-ROUTE.csv"%(DATA_NAME,NUM_FLOW,TOPO_NAME), index=False)

QUEUE = pd.DataFrame(QUEUE)
QUEUE.columns = ['id','ins_id','link','queue']
QUEUE.to_csv("RTCSA2020-%s-%d-%s-QUEUE.csv"%(DATA_NAME,NUM_FLOW,TOPO_NAME), index=False)